In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../inputs/cat-in-the-dat-train-folds.csv")

In [3]:
df.kfold.value_counts() ## 60000 dataset--5 folds == 12000 each

1    120001
0    120001
2    120000
4    119999
3    119999
Name: kfold, dtype: int64

### checking the target distribution per fold.

In [14]:
for kfold_count in range(5):
    print(f" For Kfold: {kfold_count}")
    print(df[df.kfold==kfold_count].target.value_counts())

 For Kfold: 0
0    97536
1    22465
Name: target, dtype: int64
 For Kfold: 1
0    97536
1    22465
Name: target, dtype: int64
 For Kfold: 2
0    97535
1    22465
Name: target, dtype: int64
 For Kfold: 3
0    97535
1    22464
Name: target, dtype: int64
 For Kfold: 4
0    97535
1    22464
Name: target, dtype: int64


In [14]:
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
from sklearn import linear_model


def run(fold):

    # loading the training dataset
    df = pd.read_csv("../inputs/cat-in-the-dat-train-folds.csv")

    # Selecting all the categorical columns to perform one-hot encoding
    features = [
        x for x in df.columns if x not in ["id", "target", "kfold"]
    ]

    # fill all NaN values with NONE
    for col in features:
        # converting all columns data to strings, it doesn't matter
        # as it's categorical dataset < NO Numerical values>
        df.loc[:, col] = df[col].astype(str).fillna("NONE")

    # get training dataset using folds
    df_train = df[df.kfold != fold].reset_index(drop=True)

    # get validation dataset using folds
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    ohe = preprocessing.OneHotEncoder()

    # concatenate training & validation dataset for selected columns
    full_data = pd.concat(
        [df_train[features], df_valid[features]],
        axis=0
    )
    
    ohe.fit(full_data)

    x_train = ohe.transform(df_train[features])
    print(type(x_train))

    x_valid = ohe.transform(df_valid[features])

    # initialize Logistic Regression model
    model = linear_model.LogisticRegression()
    
    # fit the model on training dataset
    model.fit(x_train, df_train.target.values)
    
    valid_preds = model.predict_proba(x_valid)[:,1]
    
    auc = metrics.roc_auc_score(df_valid.target.values, valid_preds)
    
    print(auc)

In [15]:
run(1)

<class 'scipy.sparse.csr.csr_matrix'>


C:\Users\rakes\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7859276553809572
